In [5]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt

import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [7]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [8]:
# calling API 
data=YahooFinancials('meta').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
meta_df = pd.DataFrame(data['META']['prices'])
meta_df =meta_df.drop(['high','low','open','volume','date','adjclose'], axis=1)
meta_df.rename(columns={'formatted_date':'date'},inplace=True)
meta_df['date']=pd.to_datetime(meta_df['date'])

In [9]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [10]:
# Scale data for ML engine
scaler = MinMaxScaler()
meta_df['scaled_close'] = scaler.fit_transform(np.expand_dims(meta_df['close'].values, axis=1))

In [11]:
def PrepareData(days):
    df = meta_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [12]:
PrepareData(3) # 3 days


(          close       date  scaled_close    future
 0    190.419998 2019-11-07      0.346132  0.359941
 1    190.839996 2019-11-08      0.347564  0.355577
 2    189.610001 2019-11-11      0.343370  0.355440
 3    194.470001 2019-11-12      0.359941  0.362090
 4    193.190002 2019-11-13      0.355577  0.369932
 ..          ...        ...           ...       ...
 751   95.199997 2022-11-01      0.021448  0.006410
 752   90.540001 2022-11-02      0.005558  0.026631
 753   88.910004 2022-11-03      0.000000  0.025778
 754   90.790001 2022-11-04      0.006410  0.042827
 755   96.720001 2022-11-07      0.026631  0.078290
 
 [756 rows x 4 columns], array([[0.0350871 ],
        [0.01449177],
        [0.02144779],
        [0.00555801],
        [0.        ],
        [0.00641047],
        [0.02663074],
        [0.02577829],
        [0.04282742],
        [0.07828963]], dtype=float32), array([[[0.3461315453662177, Timestamp('2019-11-07 00:00:00')],
         [0.34756366656742843, Timestamp('2019-11

In [13]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [14]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
94/94 [==============================] - 4s 9ms/step - loss: 0.0219
Epoch 2/80
94/94 [==============================] - 1s 8ms/step - loss: 0.0050
Epoch 3/80
94/94 [==============================] - 1s 8ms/step - loss: 0.0043
Epoch 4/80
94/94 [==============================] - 1s 9ms/step - loss: 0.0038
Epoch 5/80
94/94 [==============================] - 1s 12ms/step - loss: 0.0039
Epoch 6/80
94/94 [==============================] - 1s 15ms/step - loss: 0.0038
Epoch 7/80
94/94 [==============================] - 1s 12ms/step - loss: 0.0034
Epoch 8/80
94/94 [==============================] - 1s 13ms/step - loss: 0.0030
Epoch 9/80
94/94 [==============================] - 1s 13ms/step - loss: 0.0028
Epoch 10/80
94/94 [==============================] - 1s 12ms/step - loss: 0.0026
Epoch 11/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0031
Epoch 12/80
94/94 [==============================] - 1s 12ms/step - loss: 0.0024
Epoch 13/80
94/94 [======================

In [15]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (106.12$, 97.12$, 87.81$)


In [16]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
meta_lstm=pd.DataFrame()
meta_lstm['future days']=days
meta_lstm['price']=predictions
meta_lstm['ticker']='META'

meta_lstm.to_csv('meta_lstm.csv') 
files.download('meta_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>